In [10]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import TimeDistributed, LSTM, Dense, GlobalAveragePooling2D, Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.models import Model
import os
import cv2

In [3]:
# Step 1: Calculate average frame count
def get_average_frame_count(video_folders):
    frame_counts = []
    for folder in video_folders:
        frame_count = len([f for f in os.listdir(folder) if f.endswith('.jpg') or f.endswith('.png')])
        frame_counts.append(frame_count)
    return int(np.mean(frame_counts))

# Function to get list of subfolder paths
def get_subfolder_paths(folder_path):
    subfolder_paths = []
    # Iterate over all items in the directory
    for item in os.listdir(folder_path):
        item_path = os.path.join(folder_path, item)
        # Check if the item is a directory
        if os.path.isdir(item_path):
            subfolder_paths.append(item_path)
    return subfolder_paths

# Example usage:
folder_path = r"C:\Users\alosh\Downloads\FallDataset"  # Replace with the path to your folder
video_folders = get_subfolder_paths(folder_path)

 # Fill this with your actual video folder paths
AVG_FRAME_COUNT = get_average_frame_count(video_folders)

print(AVG_FRAME_COUNT)

99


In [14]:
# Step 2: Preprocess videos
def preprocess_video(folder_path, target_frames=AVG_FRAME_COUNT, target_size=(640, 480)):
    frames = []
    frame_files = sorted([f for f in os.listdir(folder_path) if f.endswith('.jpg') or f.endswith('.png')])
    
    for frame_file in frame_files:
        frame = cv2.imread(os.path.join(folder_path, frame_file))
        frame = cv2.resize(frame, target_size)
        frame = frame / 255.0  # Normalize pixel values
        frames.append(frame)
    
    # Pad or truncate
    if len(frames) < target_frames:
        frames += [np.zeros_like(frames[0])] * (target_frames - len(frames))
    elif len(frames) > target_frames:
        frames = frames[:target_frames]
    
    return np.array(frames)

# Step 3: Build the model
def build_cnn_lstm_model(input_shape):
    # Define a simple CNN model
    def cnn_model():
        inputs = tf.keras.Input(shape=input_shape[1:])
        x = Conv2D(32, (3, 3), activation='relu')(inputs)
        x = MaxPooling2D((2, 2))(x)
        x = Conv2D(64, (3, 3), activation='relu')(x)
        x = MaxPooling2D((2, 2))(x)
        x = Conv2D(64, (3, 3), activation='relu')(x)
        x = GlobalAveragePooling2D()(x)
        return Model(inputs, x)
    
    # Create our model
    inputs = tf.keras.Input(shape=input_shape)
    print(inputs)
    x = TimeDistributed(cnn_model())(inputs)
    x = LSTM(256, return_sequences=True)(x)
    x = LSTM(128)(x)
    outputs = Dense(1, activation='sigmoid')(x)
    
    model = Model(inputs, outputs)
    return model

In [15]:
# Define input shape
INPUT_SHAPE = (AVG_FRAME_COUNT, 480, 640, 3)

# Build and compile the model
model = build_cnn_lstm_model(INPUT_SHAPE)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Step 4: Training loop
def train_model(model, video_folders, epochs=10, batch_size=4):
    for epoch in range(epochs):
        print(f"Epoch {epoch+1}/{epochs}")
        for i in range(0, len(video_folders), batch_size):
            batch_folders = video_folders[i:i+batch_size]
            
            batch_data = []
            for folder in batch_folders:
                frames = preprocess_video(folder)
                batch_data.append(frames)
            
            batch_data = np.array(batch_data)
            batch_labels = np.ones(len(batch_data))  # All sequences are falls
            
            loss, accuracy = model.train_on_batch(batch_data, batch_labels)
            print(f"Batch {i//batch_size + 1} - Loss: {loss:.4f}, Accuracy: {accuracy:.4f}")

# Train the model
train_model(model, video_folders)

# Save the model
model.save('fall_recognition_model.h5')

# Function to predict fall likelihood
def predict_fall(model, video_folder):
    frames = preprocess_video(video_folder)
    frames = np.expand_dims(frames, axis=0)  # Add batch dimension
    prediction = model.predict(frames)[0]
    return prediction

# Example usage:
# fall_likelihood = predict_fall(model, 'path/to/test_video_folder')
# print(f"Fall likelihood: {fall_likelihood[0]:.2f}")

<KerasTensor shape=(None, 99, 480, 640, 3), dtype=float32, sparse=None, name=keras_tensor_197>


NotImplementedError: Exception encountered when calling TimeDistributed.call().

[1mLayer Functional should implement `def compute_output_shape(self, input_shape)`.[0m

Arguments received by TimeDistributed.call():
  • args=('<KerasTensor shape=(None, 99, 480, 640, 3), dtype=float32, sparse=None, name=keras_tensor_197>',)
  • kwargs={'mask': 'None'}